### ScratchPad for Bug
 **LangLearner Activation Stats**

https://github.com/fastai/fastai/issues/2850

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.text.all import *

In [3]:
AWD_LSTM2

fastai.text.models.awdlstm.AWD_LSTM2

In [41]:
path = untar_data(URLs.IMDB_SAMPLE)
imdb = pd.read_csv(path/'texts.csv')

In [42]:
N = 40
imdb_sample = imdb.iloc[:N]

In [62]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(512, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(512, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=512, bias=True)
    (output_dp): RNNDropout()
  )
)

In [43]:
imdb_lm = TextDataLoaders.from_df(imdb_sample, text_col='text', 
                                  is_lm=True, bs=10)

learn   = language_model_learner(imdb_lm, AWD_LSTM, 
                                 cbs = [ActivationStats],
                                 metrics=[accuracy]);

/home/user/anaconda3/envs/devfastai/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


change none


In [51]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,3.853854,3.572212,0.276754,00:15


In [54]:
as_hooks = learn.activation_stats.hooks.hooks
as_hooks

In [56]:
h_0 = as_hooks[0]
h_1 = as_hooks[-1]

In [59]:
h_0.stored is None

True

In [57]:
h_1.stored

{'mean': 0.9112761616706848,
 'std': 2.324983596801758,
 'near_zero': 0.3682360197368421}

Appears that `text.models.core.LinearDecoder.forward()` is responsible for only one layer.

in torch.nn.modules.container.Sequential.forward():

self is the sequential

`self[0]._modules['encoder']` does have hooks:
  
  ._forward_hooks: OrdereddDict of 2 fastai.callback.hook.Hook objects
  
module._modules['rnns'][0]._modules['module']

    also has forward hooks
    
Why aren't these being called?

#### Changed Learner

In [61]:
learn2   = language_model_learner(imdb_lm, AWD_LSTM, 
                                  pretrained=False,
                                 cbs = [ActivationStats],
                                 metrics=[accuracy]);

KeyError: <class 'fastai.text.models.awdlstm.AWD_LSTM'>

In [38]:
learn.activation_stats.hooks.hooks

In [40]:
aas.hooks.hooks

In [32]:
# learn2.model

In [33]:
learn2.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,5.516235,4.739337,0.193110,00:32


In [35]:
my_stats = learn2.activation_stats.stats[0]

In [36]:
list(my_stats)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'mean': 0.0017895768396556377,
  'std': 0.025393037125468254,
  'near_zero': 0.9736680772569445}]